In [3]:
from re import *

In [4]:
mail = "Bonjour, Nous souhaiterions utiliser la salle 736 le mardi 17 janvier entre 15 h et 19 h et la salle 140 le 18 janvier 9 h – 11 h."

In [26]:
#motif = (r"(?:salle+\s+\d+)|(?:Salle+\s+\d+)|\d+\s+h|(?:\d+\s+janvier)|(?:\d+\s+fevrier)|(?:\d+\s+mars)|(?:\d+\s+avril)|(?:\d+\s+mai)|(?:\d+\s+juin)|(?:\d+\s+juillet)|(?:\d+\s+août)|(?:\d+\s+septembre)|(?:\d+\s+octobre)|(?:\d+\s+novembre)|(?:\d+\s+decembre)")

In [ ]:
motif_salle = (r"(?:salle+\s+\d+)|(?:Salle+\s+\d+)")
motif_date = (r"(?:\d+\s+janvier)|(?:\d+\s+fevrier)|(?:\d+\s+mars)|(?:\d+\s+avril)|(?:\d+\s+mai)|(?:\d+\s+juin)|(?:\d+\s+juillet)|(?:\d+\s+août)|(?:\d+\s+septembre)|(?:\d+\s+octobre)|(?:\d+\s+novembre)|(?:\d+\s+decembre)")
motif_heure = (r"(?:\d+\s+Heures)|(?:\d+\s+Heure)|(?:\d+\s+heures)|(?:\d+\s+heure)|(?:\d+\s+h)|(?:\d+\s+H)|(?:\d+h)|(?:\d+H)")

motif = (r"(?:salle+\s+\d+)|(?:Salle+\s+\d+)|(?:\d+\s+janvier)|(?:\d+\s+fevrier)|(?:\d+\s+mars)|(?:\d+\s+avril)|(?:\d+\s+mai)|(?:\d+\s+juin)|(?:\d+\s+juillet)|(?:\d+\s+août)|(?:\d+\s+septembre)|(?:\d+\s+octobre)|(?:\d+\s+novembre)|(?:\d+\s+decembre)|)

In [29]:
def read_mail(s):
    motif = "(\d+\s[a-zA-Z]+)\s+[a-zA-Z]+\s(\d+)\s[a-zA-Z]\s[a-zA-Z]+\s(\d+)\s[a-zA-Z]\s[a-zA-Z]+\s[a-zA-Z]+\s[a-zA-Z]+\s([a-zA-Z]+\s\d+)"
    motif_2 = "([a-zA-Z]+\s\d+)\s+[a-zA-Z]+\s(\d+\s+[a-zA-Z]+)\s+[a-zA-Z]+\s+(\d+)\s[a-zA-Z]+\s[a-zA-Z]+\s(\d+)\s[a-zA-Z]+\s[a-zA-Z]+"
    motif_3 = "(\d+)\s[a-zA-Z]+\s[a-zA-Z]+\s(\d+)\s[a-zA-Z]+\s[a-zA-Z]+\s(\d+\s[a-zA-Z]+)\s[a-zA-Z]+\s[a-zA-Z]+\s([a-zA-Z]+\s\d+)"
    res = findall(motif, mail)
    res_2 = findall(motif_2, mail)
    res_3 = findall(motif_3, mail)
    return res , res_2 , res_3

In [30]:
print(read_mail(mail))

([], [], [('17', '15', '19 h', 'salle 140')])


In [34]:
def read_mail(s):
    motif = "[a-zA-Z]+\s(\d+)[^\d]+([\d]+)\s*([a-zA-Z]+)[^\d]+([\d]+)[^\d]+([\d]+)"
    res = findall(motif, mail)
    return res
print(read_mail(mail))

[('736', '17', 'janvier', '15', '19'), ('140', '18', 'janvier', '9', '11')]


## test de lecture email avec python

In [86]:
read_email_from_gmail()

error: b'[ALERT] Your account is not enabled for IMAP use. Please visit your Gmail settings page and enable your account for IMAP access. (Failure)'

In [84]:


ORG_EMAIL   = "@gmail.com"
FROM_EMAIL  = "marais.anthony.pro" + ORG_EMAIL
FROM_PWD    = "Viruscloud1:"
SMTP_SERVER = "imap.gmail.com"
SMTP_PORT   = 993





import smtplib
import time
import imaplib
import email

# -------------------------------------------------
#
# Utility to read email from Gmail Using Python
#
# ------------------------------------------------

def read_email_from_gmail():
    try:
        mail = imaplib.IMAP4_SSL(SMTP_SERVER)
        mail.login(FROM_EMAIL,FROM_PWD)
        mail.select('inbox')

        type, data = mail.search(None, 'ALL')
        mail_ids = data[0]

        id_list = mail_ids.split()   
        first_email_id = int(id_list[0])
        latest_email_id = int(id_list[-1])


        for i in range(latest_email_id,first_email_id, -1):
            typ, data = mail.fetch(str(i).encode(), '(RFC822)' )

            for response_part in data:
                if isinstance(response_part, tuple):
                    msg = email.message_from_string(response_part[1].decode())
                    email_subject = msg['subject']
                    email_from = msg['from']
                    print('From : ' + email_from + '\n')
                    print('Subject : ' + email_subject + '\n')

    except Exception as e:
        raise

In [1]:
import imaplib
import email
from email.header import decode_header
import webbrowser
import os
import sys


sys.stdout = open("test.txt", "w")

# account credentials
username = "marais.anthony.pro@gmail.com"
password = "Viruscloud1:"


# create an IMAP4 class with SSL 
imap = imaplib.IMAP4_SSL("imap.gmail.com")
# authenticate
imap.login(username, password)

status, messages = imap.select("INBOX")
# number of top emails to fetch
N = 4
# total number of emails
messages = int(messages[0])


for i in range(messages, messages-N, -1):
    # fetch the email message by ID
    res, msg = imap.fetch(str(i), "(RFC822)")
    for response in msg:
        if isinstance(response, tuple):
            # parse a bytes email into a message object
            msg = email.message_from_bytes(response[1])
            # decode the email subject
            subject = decode_header(msg["Subject"])[0][0]
            if isinstance(subject, bytes):
                # if it's a bytes, decode to str
                subject = subject.decode()
            # decode email sender
            From, encoding = decode_header(msg.get("From"))[0]
            if isinstance(From, bytes):
                From = From.decode(encoding)
            print("Subject:", subject)
            print("From:", From)
            # if the email message is multipart
            if msg.is_multipart():
                # iterate over email parts
                for part in msg.walk():
                    # extract content type of email
                    content_type = part.get_content_type()
                    content_disposition = str(part.get("Content-Disposition"))
                    try:
                        # get the email body
                        body = part.get_payload(decode=True).decode()
                    except:
                        pass
                    if content_type == "text/plain" and "attachment" not in content_disposition:
                        # print text/plain emails and skip attachments
                        print(body)
                    elif "attachment" in content_disposition:
                        # download attachment
                        filename = part.get_filename()
                        if filename:
                            if not os.path.isdir(subject):
                                # make a folder for this email (named after the subject)
                                os.mkdir(subject)
                            filepath = os.path.join(subject, filename)
                            # download attachment and save it
                            open(filepath, "wb").write(part.get_payload(decode=True))
            else:
                # extract content type of email
                content_type = msg.get_content_type()
                # get the email body
                body = msg.get_payload(decode=True).decode()
                if content_type == "text/plain":
                    # print only text email parts
                    print(body)
            if content_type == "text/html":
                # if it's HTML, create a new HTML file and open it in browser
                if not os.path.isdir(subject):
                    # make a folder for this email (named after the subject)
                    os.mkdir(subject)
                filename = f"{subject[:50]}.html"
                filepath = os.path.join(subject, filename)
                # write the file
                open(filepath, "w").write(body)
                # open in the default browser
                webbrowser.open(filepath)
            print("="*100)
sys.stdout.close()

# close the connection and logout
imap.close()
imap.logout()


In [25]:
file = open("/home/virus/Projet-Pedagogique/Code/test.txt"

SyntaxError: invalid character in identifier (<ipython-input-25-c648cb8439bd>, line 1)